In [1]:
##https://www.kaggle.com/datasets/noorsaeed/songs-recommendation-dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [4]:
df.shape

(57650, 4)

In [5]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [6]:
df.shape

(5000, 3)

In [7]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [8]:
df['text'][0]

"time is like a dream   and now, for a time, you are mine   let's hold fast to the dream   that tastes and sparkles like wine   who knows (who knows)   if it's real   or just something we're both dreaming of   what seems like an interlude now   could be the beginning of love   loving you   is a world that's strange   so much more than my heart can hold   loving you   makes the whole world change   loving you, i could not grow old   no, nobody knows   when love will end   so till then, sweet friend...   time is like a dream   and now, for a time, you are mine   let's hold fast to the dream   that tastes and sparkles like wine   who knows (who knows)   if it's real   or just something we're both dreaming of?   what seems like an interlude now   could be the beginning of love   what seems like an interlude now   could be the beginning of love   what seems like an interlude now   could be the beginning of love  "

In [9]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [10]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [13]:
similarity[0]

array([1.        , 0.07111849, 0.03205356, ..., 0.0231464 , 0.00240886,
       0.04274032])

In [14]:
df['song'][0]

'Interlude'

# Recommedation Function

In [17]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [18]:
recommendation('Shall We Dance')

['High',
 'Praise The War Machine',
 'The Water Is Wide',
 'There Is A Fountain',
 'The Path Part 1 - Treading Through Darkness',
 'O Come All Ye Faithful',
 'Danny Boy',
 'Amazing Grace',
 "I Can't Dance",
 'Free',
 'Dancing Girls',
 'Domino',
 'Whisper',
 'Careless Whisper',
 'Dark Star',
 "Let's Dance",
 'Down To The River To Pray',
 'Down To The River From O Brother Where Art Thou',
 'Dream Dancing',
 'Deeper']

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))